In [2]:
from pathlib import Path
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import soundfile as sf
import pandas as pd

import os 
import itertools
import re
import math
import csv
import random
from sklearn import tree
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from scipy.fftpack import fftn, ifftn, fft, ifft
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy import signal
import pickle
from sklearn.externals import joblib
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
import warnings

warnings.filterwarnings('ignore') 
df = pd.read_csv("unfilteredFeatures_aud.csv")

df = df.drop ('channel_1_medf', axis=1)
df = df.drop ('channel_2_medf', axis=1)
df = df.drop ('channel_3_medf', axis=1)
df = df.drop ('channel_4_medf', axis=1)
df = df.drop ('channel_5_medf', axis=1)
df = df.drop ('channel_6_medf', axis=1)

df = df.drop ('channel_1_var', axis=1)
df = df.drop ('channel_2_var', axis=1)
df = df.drop ('channel_3_var', axis=1)
df = df.drop ('channel_4_var', axis=1)
df = df.drop ('channel_5_var', axis=1)
df = df.drop ('channel_6_var', axis=1)

#dfs = df[(df.modeID != 'audible')]
df

,channel_1_mav,channel_1_mav2,channel_1_rms,channel_1_iav,channel_1_ssi,channel_1_wl,channel_1_iemg,channel_1_aac,channel_1_zc,channel_1_ssc,...,channel_6_iemg,channel_6_aac,channel_6_zc,channel_6_ssc,channel_6_wamp,channel_6_meanf,userID,sessionID,uttID,modeID
0,0.067106,0.065995,0.081097,224.738037,22.025275,191.597839,224.738037,0.057210,682,2043,...,300.390778,0.051220,374,1713,6,115.891113,1,1001,10010001,audible
1,0.063718,0.063090,0.075014,143.621399,12.683361,103.957947,143.621399,0.046122,288,1420,...,193.602020,0.039666,128,1199,3,114.028062,1,1001,10010002,audible
2,0.067169,0.066745,0.081422,272.503754,26.896239,227.992035,272.503754,0.056197,816,2459,...,375.475525,0.057614,520,2043,15,119.874289,1,1001,10010003,audible
3,0.065079,0.065057,0.078301,271.053162,25.535573,224.132660,271.053162,0.053813,772,2604,...,393.343292,0.064233,596,2124,21,117.951279,1,1001,10010004,audible
4,0.065251,0.065069,0.077894,306.745789,28.522983,239.254150,306.745789,0.050894,756,2905,...,435.357483,0.058570,614,2374,18,121.944233,1,1001,10010005,audible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7374,0.071839,0.070576,0.085800,134.123810,13.744313,102.125336,134.123810,0.054700,296,1161,...,146.615143,0.011380,0,1133,0,50.695711,8,8101,81010506,audible
7375,0.071289,0.070650,0.084742,139.227325,14.024961,94.420532,139.227325,0.048346,292,1141,...,153.386810,0.010563,0,1212,0,47.918460,8,8101,81010507,audible
7376,0.072052,0.071130,0.085698,122.127991,12.448364,85.883698,122.127991,0.050669,250,1014,...,133.009125,0.011079,0,1041,0,44.276136,8,8101,81010508,audible
7377,0.072907,0.072599,0.087714,142.459381,15.033598,105.453644,142.459381,0.053968,312,1176,...,153.769745,0.011253,0,1165,0,47.917805,8,8101,81010509,audible


In [3]:
def split_data_per_session(session_id):
    temp_df = df[(df.sessionID == session_id)]
    data = temp_df.copy()
    data = shuffle(data)
    y = data['modeID']
    data.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return train_test_split(data, y, test_size=0.2)

def split_data_session(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['sessionID'].isin(train_user_ids)]
    test_df = data.loc[data['sessionID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['userID']
    y_test = X_test['userID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

def split_data_utterance(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['userID'].isin(train_user_ids)]
    test_df = data.loc[data['userID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['userID']
    y_test = X_test['userID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

def split_data(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['userID'].isin(train_user_ids)]
    test_df = data.loc[data['userID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

In [4]:
def lda():
    ids = [1001,1002,1003,2001,2002,2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,2011, 2012, 2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,
        2023,2024,2025,2026,2027,2028, 2029, 2030,2031, 2032,2101,8001,8002,8003,8004,8005,8006,8007,8008,8009, 8010, 8011,8012,8013,8014,8015, 8016, 8017, 8018,8019, 8101]
    scores = np.array([])
    for x in range (len(ids)):
        test_ids = []
        test_ids = ids[x:x+1]
        train_ids = np.delete(ids,x)
        #print (test_ids)
        X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
        k_clf = LinearDiscriminantAnalysis()
        k_clf.fit(X_train, y_train)
        score = k_clf.score(X_test, y_test)
        sscore = np.array([score])
        scores = np.concatenate([scores,sscore],axis=0)
        print("Accuracy for session " + str(test_ids) +" is "+str(score))
    print ("Durchschnitt: ")
    print((scores.mean()))

In [5]:
lda()

Accuracy for session [1001] is 0.9333333333333333
Accuracy for session [1002] is 0.9866666666666667
Accuracy for session [1003] is 0.9933333333333333
Accuracy for session [2001] is 0.9866666666666667
Accuracy for session [2002] is 1.0
Accuracy for session [2003] is 1.0
Accuracy for session [2004] is 0.98
Accuracy for session [2005] is 1.0
Accuracy for session [2006] is 1.0
Accuracy for session [2007] is 1.0
Accuracy for session [2008] is 0.9933333333333333
Accuracy for session [2009] is 1.0
Accuracy for session [2010] is 1.0
Accuracy for session [2011] is 1.0
Accuracy for session [2012] is 1.0
Accuracy for session [2013] is 1.0
Accuracy for session [2014] is 1.0
Accuracy for session [2015] is 1.0
Accuracy for session [2016] is 1.0
Accuracy for session [2017] is 1.0
Accuracy for session [2018] is 1.0
Accuracy for session [2019] is 1.0
Accuracy for session [2020] is 1.0
Accuracy for session [2021] is 1.0
Accuracy for session [2022] is 1.0
Accuracy for session [2023] is 1.0
Accuracy for s